In [ ]:
# Instalando o upgrade do pip
# Instalando a biblioteca pymssql

!pip install --upgrade pip
!pip install pymssql
!pip install adal

In [ ]:
# Importando Pandas, pymssql e adal

import pandas as pd
import pymssql
import adal

resource_app_id_url = "https://database.windows.net/"

In [ ]:
# Buscando os dados da API e salvando em CSV

url = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial=%2701-01-2019%27&@dataFinalCotacao=%2712-31-2025%27&$top=9000&$format=text/csv&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao"
csv = pd.read_csv(url)
csv.head()

In [ ]:
#Criando a conexão

service_principal_id = dbutils.secrets.get(scope = "scope-adb-estudos", key = "app-reg-adb")
service_principal_secret = dbutils.secrets.get(scope = "scope-adb-estudos", key = "app-user-databricks")
tenant_id = "b4920c82-7581-491a-9dab-cd2ade2f3ebd"
authority = "https://login.windows.net/" + tenant_id

In [ ]:
azure_sql_url = "jdbc:sqlserver://sql-estudo.database.windows.net"
database_name = "db-estudos"
db_table = "[dolar_marcus_araujo].[dolar_stage_marcus_araujo02]"
encrypt = "true"
host_name_in_certificate = "*.database.windows.net"

In [ ]:
context = adal.AuthenticationContext(authority)
token = context.acquire_token_with_client_credentials(resource_app_id_url, service_principal_id, service_principal_secret)
access_token = token["accessToken"]

In [ ]:
# Gerando DataFrame

emperorDf = spark.read \
             .format("jdbc") \
             .option("url", azure_sql_url) \
             .option("dbtable", db_table) \
             .option("databaseName", database_name) \
             .option("accessToken", access_token) \
             .option("encrypt", "true") \
             .option("hostNameInCertificate", "*.database.windows.net") \
             .load()           

display(emperorDf)  

In [ ]:
df = pd.DataFrame(csv)
display(df)

In [ ]:
#Tipo de dados em cada coluna

df.dtypes

In [ ]:
# Valores dos dados

df.values

In [ ]:
# Limpeza da tabela antes da inserção de dados

com02 = """truncate table [dolar_marcus_araujo].[dolar_stage_marcus_araujo02]"""

In [ ]:
# Query para a inserção de dados da API para a tabela stage

com = """INSERT INTO [dolar_marcus_araujo].[dolar_stage_marcus_araujo02] VALUES(%s, %s, %s);"""
sql_data = tuple(map(tuple, df.values)) # Convertendo arrays em tuplas
print('Dados inseridos corretamente na tabela stage')